In [1]:
###

In [1]:
import inspect
import pptx
from pptx import Presentation

def get_module_documentation(module, depth=0, max_depth=3):
    """Extract documentation from a module recursively"""
    if depth > max_depth:
        return {}
    
    docs = {}
    for name, obj in inspect.getmembers(module):
        # Filter out private attributes and methods
        if name.startswith('_'):
            continue
            
        # Get documentation
        if inspect.isclass(obj):
            class_doc = inspect.getdoc(obj) or "No documentation"
            method_docs = {}
            
            # Get methods and their documentation
            for method_name, method in inspect.getmembers(obj, inspect.isfunction):
                if not method_name.startswith('_'):
                    method_docs[method_name] = inspect.getdoc(method) or "No documentation"
            
            docs[name] = {
                "type": "class",
                "doc": class_doc,
                "methods": method_docs
            }
        
        # For modules, recurse deeper
        elif inspect.ismodule(obj) and obj.__name__.startswith('pptx'):
            docs[name] = {
                "type": "module",
                "doc": inspect.getdoc(obj) or "No documentation",
                "members": get_module_documentation(obj, depth+1, max_depth)
            }
    
    return docs

# Extract docs from pptx module
api_docs = get_module_documentation(pptx)

# Format as string for prompt
import json
api_docs_str = json.dumps(api_docs, indent=2)
print(api_docs_str)
#save to file

def save(api_docs_str):
    with open("api_docs.json", "w") as f:
        f.write(api_docs_str)    
    print("API docs saved to api_docs.json")

save(api_docs_str)


{
  "NotesMasterPart": {
    "type": "class",
    "doc": "Notes master part.\n\nCorresponds to package file `ppt/notesMasters/notesMaster1.xml`.",
    "methods": {
      "drop_rel": "Remove relationship identified by `rId` if its reference count is under 2.\n\nRelationships with a reference count of 0 are implicit relationships. Note that only XML\nparts can drop relationships.",
      "get_image": "Return an |Image| object containing the image related to this slide by *rId*.\n\nRaises |KeyError| if no image is related by that id, which would generally indicate a\ncorrupted .pptx file.",
      "get_or_add_image_part": "Return `(image_part, rId)` pair corresponding to `image_file`.\n\nThe returned |ImagePart| object contains the image in `image_file` and is\nrelated to this slide with the key `rId`. If either the image part or\nrelationship already exists, they are reused, otherwise they are newly created.",
      "load_rels_from_xml": "load _Relationships for this part from `xml_rels`.